In [10]:
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from langchain_groq import ChatGroq
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import os
import fitz
import pdfplumber
import json
import ollama
import json
from pymongo import MongoClient

In [12]:
connection_string = "mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.2.10"
client = MongoClient(connection_string)
db = client['Document_Reader']  
collection = db['doc_to_json_1']  

In [6]:
documents_folder = 'documents'  # Change this to your documents folder path
document_files = [f for f in os.listdir(documents_folder) if os.path.isfile(os.path.join(documents_folder, f))]
document_files = ['COSCO.pdf']
document_path = os.path.join(documents_folder, document_files[0])


sample_prompt = """
Your task is to extract the below fields in the same format from the text mentioned after format:
{{
    "fileName": "",
    "orgName": "",
    "orderId": "",
    "ocrBucketName": "",
    "docType": "",
    "bookingDetails": {{
        "carrierBookingNum": "",
        "billOfLadingNum": "",
        "referenceNumber": "",
        "carrierName": "",
        "originServiceMode": "",
        "destinationServiceMode": "",
        "placeOfOrigin": "",
        "placeOfDelivery": "",
        "portOfLoad": "",
        "portOfDischarge": "",
        "temperature": {{
            "unit": "",
            "value": ""
        }},
        "humidity": "",
        "gensetRequired": "",
        "travelType": "",
        "containerDetails": {{
            "code": "",
            "quantity": ""
        }},
        "weight": {{
            "unit": "",
            "value": ""
        }},
        "commodityType": "",
        "hazardous": {{
            "hazCode": "",
            "imoClassType": "",
            "packageCount": "",
            "packageType": "",
            "packageGroup": ""
        }},
        "voyageInfo": {{
            "vesselName": "",
            "voyageNumber": "",
            "imoNumber": "",
            "departureEstimated": "",
            "arrivalEstimated": ""
        }}
    }},
    "shipmentDetails": {{
        "shipmentDate": "",
        "containerPickUpDate": "",
        "railCutOffDate": "",
        "portCutOffDate": "",
        "vgmCutoffDate": "",
        "portOpenDate": "",
        "siCutOffDate": "",
        "shipOnBoardDate": "",
        "vent": ""
    }},
    "containers": [{{
        "containerId": "",
        "type": ""
    }}, {{
        "containerId": "",
        "type": ""
    }}]
}}
    
Text for information extraction:

{text}

Make sure to format the result in the specified format.

Just give the JSON result. No extra info
"""


In [7]:
def extract_text_from_pdf_direct(state):
    pdf_path=state.get("file_path")
    all_text = []

    # Open the PDF file
    with pdfplumber.open(pdf_path) as pdf:
        # Loop through each page
        for page in pdf.pages:
            # Extract text from the current page
            page_text = page.extract_text()
            if page_text:
                all_text.append(page_text)

    # Join all text from all pages
    complete_text = "\n".join(all_text)
    return {'response':complete_text}

In [15]:
prompt = PromptTemplate(template=sample_prompt)
llm = ChatGroq(model="llama3-70b-8192",groq_api_key="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")
class GraphState(TypedDict):
    file_path: Optional[str]
    response: Optional[str]
wf = StateGraph(GraphState)
def process_text(state):
    current_response = state.get('response')
    chain = prompt | llm 
    result = chain.invoke({"text": current_response})
    return {'response': result}

wf.add_node("node_1", extract_text_from_pdf_direct)
wf.add_node("node_2", process_text)
wf.set_entry_point("node_1")
wf.add_edge("node_1", "node_2")
wf.set_finish_point("node_2")

app = wf.compile()



documents_folder = 'documents'  # Change this to your documents folder path
document_files = [f for f in os.listdir(documents_folder) if os.path.isfile(os.path.join(documents_folder, f))]
for i in range(len(document_files)):
    if (".pdf" in document_files[i]):
        document_path = os.path.join(documents_folder, document_files[i])
        output = app.invoke({'file_path': document_path})
        res=output['response'].content
        res=json.loads(res)
        res['fileName']=document_files[i]
        collection.insert_one(res)

In [ ]:
import json
from pymongo import MongoClient

connection_string = "mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.2.10"


# Connect to MongoDB
client = MongoClient(connection_string)

db = client['Document_Reader']  # Replace 'your_database' with the name of your database
collection = db['doc_to_json_1']  # Replace 'your_collection' with the name of your collection
if isinstance(res, list):
    collection.insert_many(res)
else:
    collection.insert_one(res)

